Togeher AI setup for a low cost open source way to validate GenAI prompts.

In [ ]:
!pip install together

In [ ]:
from together import Together

def send_prompt(messages: []):
    client = Together(api_key="<API_KEY>")
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=messages,
    )
    return response.choices[0].message.content

def create_prompt(prompt: str):
    return [{"role": "user", "content": prompt}]

Zero-Shot Learning: This involves giving the AI a task without any prior examples. You describe what you want in detail, assuming the AI has no prior knowledge of the task.

In [ ]:
# Zero-Shot
zero_shot_prompt = """
    Explain what a large language model is."""

prompt = create_prompt(zero_shot_prompt)

print(send_prompt(prompt))

One-Shot Learning: You provide one example along with your prompt. This helps the AI understand the context or format you’re expecting.

In [ ]:
one_shot_prompt = """
    A Foundation Model in AI refers to a model like GPT-3,
    which is trained on a large dataset and can be adapted to various tasks.
    Explain what BERT is in this context."""

prompt = create_prompt(one_shot_prompt)

print(send_prompt(prompt))

Few-Shot Learning: This involves providing a few examples (usually 2–5) to help the AI understand the pattern or style of the response you’re looking for.

In [ ]:
few_shot_prompt = """
    Foundation Models such as GPT-3 are used for natural language
    processing, while models like DALL-E are used for image generation.
    How are Foundation Models used in the field of robotics?"""

prompt = create_prompt(few_shot_prompt)

print(send_prompt(prompt))

Instruction Prompting: Embed explicit task steps within the prompt for the AI to follow.

In [ ]:
Instruction_prompt = """
    Read the following wikipedia article. remove the [1], [2], [3] from the article.

    The largest and most capable LLMs are generative pretrained transformers (GPTs), which are largely used in generative chatbots such as ChatGPT, Gemini or Claude. 
    LLMs can be fine-tuned for specific tasks or guided by prompt engineering.[1] These models acquire predictive power regarding syntax, semantics, and ontologies[2] 
    inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.[3]?"""

prompt = create_prompt(Instruction_prompt)

print(send_prompt(prompt))

Chain-of-Thought Prompting: Here, you ask the AI to detail its thought process step-by-step. This is particularly useful for complex reasoning tasks.

In [ ]:
chain_of_thought_prompt = """
    Describe the process of developing a Foundation Model in AI,
    from data collection to model training."""

prompt = create_prompt(chain_of_thought_prompt)

print(send_prompt(prompt))

Iterative Prompting: This is a process where you refine your prompt based on the outputs you get, slowly guiding the AI to the desired answer or style of answer.

In [ ]:
# Store previous messages for better context
messages = []
iterative_prompt = """
    Tell me about the latest developments in Foundation Models in AI."""

messages += create_prompt(iterative_prompt)

first_response = send_prompt(messages)
print(first_response)

messages.append({"role": "assistant","content":first_response["message"]["content"]})

refined_prompt = """
    Can you provide more details about these improvements in multi-modal learning within Foundation Models?"""

messages += create_prompt(refined_prompt)
print("\n\n----------------------------------------------------------\n\n")
print(send_prompt(messages))

Negative Prompting: In this method, you tell the AI what not to do. For instance, you might specify that you don’t want a certain type of content in the response.

In [ ]:
negative_prompt = """
    Explain the concept of Foundation Models in AI without mentioning natural language processing or NLP."""

prompt = create_prompt(negative_prompt)

print(send_prompt(prompt))

Hybrid Prompting: Combining different methods, like few-shot with chain-of-thought, to get more precise or creative outputs.

In [ ]:
hybrid_prompt = """
    Like GPT-3, which is a versatile model used in various language tasks, 
    explain how Foundation Models are applied in other domains of AI, such as computer vision."""

prompt = create_prompt(hybrid_prompt)

print(send_prompt(prompt))

 Prompt Chaining: Breaking down a complex task into smaller prompts and then chaining the outputs together to form a final response.

In [ ]:
messages = []
chaining_prompt  = """
    List some examples of Foundation Models in AI."""

messages += create_prompt(chaining_prompt)

print(send_prompt(messages))

messages.append({"role": "assistant","content":first_response["message"]["content"]})

chaining_content_prompt = """
    Choose one of these models and explain its foundational role in AI development."""

messages += create_prompt(chaining_content_prompt)
print("\n\n----------------------------------------------------------\n\n")
print(send_prompt(messages))

Tree of thought / Self-Consistency- breakdown the task into steps and ask multiple sources. choose the best source and then continue the instructions.

In [ ]:
tree_of_thought_prompt = """
    Imagine three different experts are answering this question. 
    All experts will write down 1 step of their thinking, then share it with the group. 
    Then all experts will go on to the next step, etc. 
    If any expert realises they're wrong at any point then they leave. 
    The question is: 
    48 people are riding a bus. On the first stop, 8 passengers get off, and 5 times as many people as the number who got off from the bus get into the bus. On the second stop 21, passengers get off and 3 times fewer passengers get on. How many passengers are riding the bus after the second stop?
    """

prompt = create_prompt(tree_of_thought_prompt)

print(send_prompt(prompt))

Directional stimulus prompt - give the AI a hint about what you want to help guide the AI to the correct answer

In [ ]:
directional_stimulus_prompt = """
    if 5+5=10, 8+2=10, and 9+1=10 what does 7+3=?
    Summarize the above in 2-3 sentences based on the hint. 
    Hint: The answer is 10.
    """

prompt = create_prompt(directional_stimulus_prompt)

print(send_prompt(prompt))



Chain of density - take all the text and create a summary. use the prompt to check the summary and make sure everything was covered. continue to do this until we are satisfied.

In [ ]:
ARTICLE = """
A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text, designed for natural language processing tasks, especially language generation. 
    The largest and most capable LLMs are generative pretrained transformers (GPTs), which are largely used in generative chatbots such as ChatGPT, Gemini or Claude. LLMs can be fine-tuned for specific tasks or 
    guided by prompt engineering.[1] These models acquire predictive power regarding syntax, semantics, and ontologies[2] inherent in human language corpora, but they also inherit inaccuracies and biases present 
    in the data they are trained in.[3]
    """
chain_of_density_prompt = f"""
    Article: {ARTICLE}
    You will generate increasingly concise, entity-dense summaries of the above Article.
    Repeat the following 2 steps 5 times.
    Step 1. Identify 1-3 informative Entities (". " delimited) from the Article which are missing from the previously generated summary.
    Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.
    A Missing Entity is:
    - Relevant: to the main story.
    - Specific: descriptive yet concise (5 words or fewer).
    - Novel: not in the previous summary.
    - Faithful: present in the Article.
    - Anywhere: located anywhere in the Article.
    Guidelines:
    - The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
    - Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
    - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
    - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
    - Missing entities can appear anywhere in the new summary.
    - Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

    Remember, use the exact same number of words for each summary.

    Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
    """

prompt = [{"role": "user", "content": chain_of_density_prompt}]

print(send_prompt(prompt))